NHÓM 8: VŨ XUÂN CHUNG, NGÔ NHẬT MINH, NGUYỄN TRUNG PHÚC

Event time

Event time là một chủ đề quan trọng và riêng biệt, bởi vì Dstream API của Spark không hỗ trợ truy cập thông tin liên quan đến event-time. Ở mức cao hơn, trong hệ thống xử lý luồng stream, có 2 thời điểm thích hợp cho 1 sự kiện xảy ra: thời điểm ngay lúc nó xảy ra (event time) và thời điểm nó đã được xử lý xong hoặc chạm đến hệ thống streamprocessing.

Event-time chính là thời gian được nhúng từ chính sự kiện xảy ra đó. Thông thương, mặc dù không bắt buộc, thời gian của sự kiện đó vẫn xảy ra. Điều này rất quan trọng vì nó cung cấp phương tiện mạnh mẽ đẻ so sánh 2 sự kiện với nhau. Thách thức ở đây là sự kiện xảy ra có thể bị trễ và không theo thứ tự. Điều này đồng nghĩa với việc hệ thống xử lý luồng stream cần xử lý được những event bị trễ hoặc không theo thử tự.

Processing Time

Processing time là thời gian mà hệ thống stream-processing thực tế nhận được dữ liệu. Nó thường ít quan trọng hơn event-time để dùng vì khi truy cập, phần lớn chỉ là chi tiết implementation. Thường Processing Time không thể bị sai thứ tự vì nó là một thuộc tính của một hệ thống trực tuyến tại thời điểm nhất định.  

Stateful processing

Một chủ đề khác chúng ta cần đề cập trong chapter này là xử  lý trạng thái. Xử lý trạng thái chỉ cần thiết khi cần sử dụng hoặc update một thông tin trung gian trong một thời gian dài. Điều này có thể xẩy ra khi bạn sử dung event-time hoặc khi bạn thực hiện aggregation trên khóa của dữ liệu, cho dù nó có bao gồm event-time hay không.

Hầu hết, khi bạn thực hiện các hoạt động trạng thái, Spark xử lý toàn bộ những điều phức tạp này giúp bạn. Ví dụ, khi bạn chỉ định một nhóm, Structure Stream sẽ bảo trì và cập nhật dữ liệu giúp bạn. Bạn chỉ cần tập trung vào logic. Khi thực hiện một hoạt động trạng thái, Spark sẽ lưu các thông tin trung gian vào kho lưu trữ trạng thái. Kho lưu trữ này của Spark là bộ nhớ trong được tạo ra nhằm chịu lỗi bằng cách lưu các trạng thái trung gian vào checkpoint.

Arbitrary stateful processing

Khả năng xử lý trạng thái được mô tả ở trên đủ để giải quyết nhiều vấn đề về stream.  Tuy nhiên đôi khi bạn cần kiểm soát được trạng thái nào nên được lưu trữ, cập nhật và khi nào thì nên xóa nó, một cách rõ ràng hay thông qua một time-out.
Điều này được gọi là xử lý trạng thái tùy ý, và về cơ bản, Spark cho phép bạn lưu trữ bất cứ thông tin nào bạn muốn trong suốt quá trình xử lý luồng. Điều này cung cấp tính linh hoạt và sức mạnh to lớn và cho phép một số logic nghiệp vụ phức tạp được xử lý khá dễ dàng. Giống như chúng tôi đã làm trước đây, hãy dựa vào một số ví dụ: 

- Bạn muốn ghi lại thông tin về các phiên của người dùng trên một trang web thương mại điện tử. Ví dụ: bạn có thể muốn theo dõi những trang mà người dùng truy cập trong suốt phiên này để đưa ra các đề xuất trong thời gian thực trong phiên tiếp theo của họ.  Đương nhiên, các phiên này có thời gian bắt đầu và dừng hoàn toàn tùy ý dành riêng cho người dùng đó.
-Công ty của bạn muốn báo cáo về lỗi trong ứng dụng web nhưng chỉ khi có năm sự kiện xảy ra trong phiên của người dùng.  Bạn có thể làm điều này với các cửa sổ dựa trên số lượng chỉ phát ra một kết quả nếu năm sự kiện thuộc một số loại xảy ra.
-Bạn muốn loại bỏ các bản ghi trùng lặp theo thời gian.  Để làm như vậy, bạn sẽ cần phải theo dõi mọi bản ghi mà bạn thấy trước khi sao chép nó.

Bây giờ chúng tôi đã giải thích các khái niệm cốt lõi mà chúng tôi sẽ cần trong chương này, hãy bao gồm tất cả những điều này với một số ví dụ mà bạn có thể làm theo và giải thích một số lưu ý quan trọng mà bạn cần xem xét khi xử lý  cách thức.

In [2]:
!pip install pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", 5) 
# spark.sql.shuffle.partitions Configures the number of partitions to use when shuffling data for joins or aggregations.
static = spark.read.json("/content/drive/MyDrive/Colab Notebooks/SparkTutorial/spark-lab/data/activity-data")

In [ ]:
streaming = spark.readStream.schema(static.schema).option("maxFilesPerTrigger",10)\
            .json("/content/drive/MyDrive/Colab Notebooks/SparkTutorial/spark-lab/data/activity-data")
streaming.printSchema()

root
 |-- Arrival_Time: long (nullable = true)
 |-- Creation_Time: long (nullable = true)
 |-- Device: string (nullable = true)
 |-- Index: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- User: string (nullable = true)
 |-- gt: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)



Trong tập dữ liệu này, có hai cột dựa trên thời gian.  Cột Creation_Time xác định thời điểm một sự kiện được tạo, trong khi Arrival_Time xác định thời điểm một sự kiện chạm vào máy chủ của chúng tôi ở đâu đó ngược dòng.  Chúng tôi sẽ sử dụng Creation_Time trong chương này.  Ví dụ này đọc từ một tệp nhưng, như chúng ta đã thấy trong chương trước, sẽ đơn giản để thay đổi nó thành Kafka nếu bạn đã có một cụm đang hoạt động.

Windows on Event Time

Bước đầu tiên trong phân tích thời gian sự kiện là chuyển đổi cột dấu thời gian thành loại dấu thời gian Spark SQL thích hợp.  Cột hiện tại của chúng tôi là nano giây đơn thời gian (được biểu thị là dài), do đó chúng tôi sẽ phải thực hiện một chút thao tác để đưa nó vào định dạng thích hợp:

In [ ]:
withEventTime = streaming.selectExpr("*",\
"cast(cast(Creation_Time as double)/1000000000 as timestamp) as event_time")

Chúng tôi hiện đã sẵn sàng để thực hiện các hoạt động tùy ý vào thời gian diễn ra sự kiện!  Lưu ý rằng trải nghiệm này giống như cách chúng tôi thực hiện trong các hoạt động hàng loạt — không có API hoặc DSL đặc biệt.  Chúng tôi chỉ sử dụng các cột, giống như chúng tôi có thể làm theo lô, tổng hợp và chúng tôi đang làm việc với thời gian sự kiện.

Tumbling Windows

Thao tác đơn giản nhất chỉ đơn giản là đếm số lần xuất hiện của một sự kiện trong một cửa sổ nhất định.  Hình 22-2 mô tả quá trình khi thực hiện một phép tổng đơn giản dựa trên dữ liệu đầu vào và một khóa.  Chúng tôi đang thực hiện tổng hợp các khóa trong một khoảng thời gian.  Chúng tôi cập nhật bảng kết quả (tùy thuộc vào chế độ đầu ra) khi mọi trình kích hoạt chạy, bảng này sẽ hoạt động trên dữ liệu nhận được kể từ lần kích hoạt cuối cùng.  Trong trường hợp tập dữ liệu thực tế của chúng tôi (và Hình 22-2), chúng tôi sẽ làm như vậy trong các cửa sổ 10 phút mà không có bất kỳ sự chồng chéo nào giữa chúng (mỗi và chỉ một sự kiện có thể rơi vào một cửa sổ).  Điều này cũng sẽ cập nhật theo thời gian thực, có nghĩa là nếu các sự kiện mới được thêm ngược dòng vào hệ thống của chúng tôi, Phát trực tiếp có cấu trúc sẽ cập nhật các số lượng đó cho phù hợp.  Đây là chế độ đầu ra hoàn chỉnh, Spark sẽ xuất toàn bộ bảng kết quả bất kể chúng ta đã xem toàn bộ tập dữ liệu hay chưa:

In [ ]:
from pyspark.sql.functions import window, col
withEventTime.groupBy(window(col("event_time"), "10 minutes")).count()\
.writeStream\
.queryName("events_per_window")\
.format("memory")\
.outputMode("complete")\
.start()

Bây giờ chúng tôi đang viết ra bộ nhớ chìm trong bộ nhớ để gỡ lỗi, vì vậy chúng tôi có thể truy vấn nó bằng SQL sau khi chúng tôi có luồng chạy:

In [ ]:
spark.sql("SELECT * FROM events_per_window").printSchema()

root
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- count: long (nullable = false)



In [ ]:
spark.sql("SELECT * FROM events_per_window").show()

NameError: ignored

In [ ]:
spark.sql("SELECT * FROM events_per_window").head(5)

[Row(window=Row(start=datetime.datetime(2015, 2, 24, 11, 50), end=datetime.datetime(2015, 2, 24, 12, 0)), count=150773),
 Row(window=Row(start=datetime.datetime(2015, 2, 24, 13, 0), end=datetime.datetime(2015, 2, 24, 13, 10)), count=133323),
 Row(window=Row(start=datetime.datetime(2015, 2, 23, 12, 30), end=datetime.datetime(2015, 2, 23, 12, 40)), count=100853),
 Row(window=Row(start=datetime.datetime(2015, 2, 23, 10, 20), end=datetime.datetime(2015, 2, 23, 10, 30)), count=99178),
 Row(window=Row(start=datetime.datetime(2015, 2, 24, 12, 30), end=datetime.datetime(2015, 2, 24, 12, 40)), count=125679)]

Lưu ý rằng window thực sự là một cấu trúc (một kiểu phức tạp).  Sử dụng nó, chúng tôi có thể truy vấn cấu trúc này cho thời gian bắt đầu và kết thúc của một cửa sổ cụ thể.  Điều quan trọng là chúng ta cũng có thể thực hiện tổng hợp trên nhiều cột, bao gồm cả cột thời gian sự kiện.  Giống như chúng ta đã thấy ở chương trước, chúng ta thậm chí có thể thực hiện các phép gộp này bằng các phương pháp như khối lập phương.  Mặc dù chúng tôi sẽ không lặp lại thực tế rằng chúng tôi có thể thực hiện tổng hợp nhiều khóa bên dưới, nhưng điều này áp dụng cho bất kỳ tổng hợp kiểu cửa sổ nào (hoặc tính toán trạng thái) mà chúng tôi muốn:

In [ ]:
from pyspark.sql.functions import window, col
withEventTime.groupBy(window(col("event_time"), "10 minutes"), "User").count()\
.writeStream\
.queryName("events_per_window_2")\
.format("memory")\
.outputMode("complete")\
.start()

Sliding windows

Ví dụ trước là số đếm đơn giản trong một cửa sổ nhất định.  Một cách tiếp cận khác là chúng ta có thể tách cửa sổ từ thời điểm bắt đầu của cửa sổ.  Hình 22-3 minh họa ý của chúng tôi.  Trong hình, chúng tôi đang chạy một cửa sổ trượt qua đó chúng tôi nhìn vào khoảng tăng một giờ, nhưng chúng tôi muốn có trạng thái sau mỗi 10 phút.  Điều này có nghĩa là chúng tôi sẽ cập nhật các giá trị theo thời gian và sẽ bao gồm dữ liệu giờ cuối cùng.  Trong ví dụ này, chúng tôi có các cửa sổ 10 phút, bắt đầu sau mỗi năm phút.  Do đó mỗi sự kiện sẽ rơi vào hai cửa sổ khác nhau.  Bạn có thể điều chỉnh thêm tùy theo nhu cầu của mình:

In [ ]:
from pyspark.sql.functions import window, col
withEventTime.groupBy(window(col("event_time"), "10 minutes", "5 minutes"))\
.count()\
.writeStream\
.queryName("events_per_window_3")\
.format("memory")\
.outputMode("complete")\
.start()

Đương nhiên, chúng ta có thể truy vấn bảng trong bộ nhớ:


In [ ]:
spark.sql("SELECT * FROM events_per_window_3").head(3)

[Row(window=Row(start=datetime.datetime(2015, 2, 23, 14, 15), end=datetime.datetime(2015, 2, 23, 14, 25)), count=67368),
 Row(window=Row(start=datetime.datetime(2015, 2, 24, 11, 50), end=datetime.datetime(2015, 2, 24, 12, 0)), count=94302),
 Row(window=Row(start=datetime.datetime(2015, 2, 24, 13, 0), end=datetime.datetime(2015, 2, 24, 13, 10)), count=83399)]

Handling Late Data with Watermarks

Các ví dụ trước rất tuyệt vời, nhưng chúng có một lỗ hổng.  Chúng tôi không bao giờ chỉ định trễ bao lâu để xem dữ liệu.  Điều này có nghĩa là Spark sẽ cần phải lưu trữ dữ liệu trung gian đó mãi mãi vì chúng tôi chưa bao giờ chỉ định hình mờ hoặc thời điểm mà chúng tôi không mong đợi thấy thêm bất kỳ dữ liệu nào.  Điều này áp dụng cho tất cả các xử lý trạng thái hoạt động vào thời gian sự kiện.  Chúng tôi phải chỉ định hình mờ này để xóa dữ liệu trong luồng (và do đó, trạng thái) để chúng tôi không áp đảo hệ thống trong một thời gian dài.  Cụ thể, hình mờ là khoảng thời gian sau một sự kiện nhất định hoặc một tập hợp các sự kiện mà sau đó chúng tôi không mong đợi thấy thêm bất kỳ dữ liệu nào từ thời điểm đó.

In [ ]:
from pyspark.sql.functions import window, col
withEventTime\
.withWatermark("event_time", "30 minutes")\
.groupBy(window(col("event_time"), "10 minutes", "5 minutes"))\
.count()\
.writeStream\
.queryName("events_per_window_4")\
.format("memory")\
.outputMode("complete")\
.start()

Nó khá tuyệt vời, nhưng hầu như không có gì thay đổi về truy vấn của chúng tôi.  Về cơ bản, chúng tôi chỉ thêm một cấu hình khác.  Bây giờ, Phát trực tuyến có cấu trúc sẽ đợi cho đến 30 phút sau dấu thời gian cuối cùng của cửa sổ luân phiên kéo dài 10 phút này trước khi hoàn tất kết quả của cửa sổ đó.  Chúng tôi có thể truy vấn bảng của mình và xem kết quả trung gian bởi vì chúng tôi đang sử dụng chế độ hoàn chỉnh — chúng sẽ được cập nhật theo thời gian.  Trong chế độ nối thêm, thông tin này sẽ không được xuất cho đến khi cửa sổ đóng lại.  Tại thời điểm này, bạn thực sự biết tất cả những gì bạn cần biết về việc xử lý dữ liệu trễ.  Spark thực hiện tất cả những công việc nặng nhọc giúp bạn.  Chỉ để củng cố quan điểm, nếu bạn không nói rõ bạn nghĩ mình sẽ nhìn thấy dữ liệu muộn bao lâu thì Spark sẽ duy trì dữ liệu đó trong bộ nhớ mãi mãi.  Việc chỉ định hình mờ cho phép nó giải phóng những đối tượng đó khỏi bộ nhớ, cho phép luồng của bạn tiếp tục chạy trong một thời gian dài.

Dropping Duplicates in a Stream

Một trong những hoạt động khó khăn hơn trong hệ thống ghi từng lần là xóa các bản sao khỏi luồng.  Hầu như theo định nghĩa, bạn phải thao tác trên một loạt bản ghi tại một thời điểm để tìm các bản sao — có một chi phí phối hợp cao trong hệ thống xử lý.  Nhân bản là một công cụ quan trọng trong nhiều ứng dụng, đặc biệt khi các thông điệp có thể được gửi nhiều lần bởi các hệ thống ngược dòng.  Một ví dụ hoàn hảo về điều này là các ứng dụng Internet of Things (IoT) có các nhà sản xuất thượng nguồn tạo ra các thông điệp trong môi trường mạng không ổn định và cùng một thông điệp có thể được gửi nhiều lần.  Các ứng dụng và tổng hợp hạ nguồn của bạn sẽ có thể giả định rằng chỉ có một trong mỗi thông báo.  Về cơ bản, Phát trực tiếp có cấu trúc giúp dễ dàng lấy các hệ thống thông báo cung cấp ít nhất ngữ nghĩa và chuyển đổi chúng thành chính xác một lần bằng cách loại bỏ các thông báo trùng lặp khi chúng đến, dựa trên các khóa tùy ý.  Để loại bỏ dữ liệu trùng lặp, Spark sẽ duy trì một số khóa do người dùng chỉ định và đảm bảo rằng các khóa trùng lặp được bỏ qua.

WARNING

Giống như các ứng dụng xử lý trạng thái khác, bạn cần chỉ định hình mờ để đảm bảo rằng trạng thái được duy trì không phát triển vô hạn trong quá trình phát trực tiếp của bạn.  Hãy bắt đầu quá trình khử trùng lặp.  Mục tiêu ở đây sẽ là loại bỏ số lượng sự kiện trên mỗi người dùng bằng cách loại bỏ các sự kiện trùng lặp.  Lưu ý rằng bạn cần chỉ định cột thời gian sự kiện như một cột trùng lặp cùng với cột bạn nên loại bỏ trùng lặp.  Giả định cốt lõi là các sự kiện trùng lặp sẽ có cùng dấu thời gian cũng như mã định danh.  Trong mô hình này, các hàng có hai dấu thời gian khác nhau là hai bản ghi khác nhau:

In [ ]:
from pyspark.sql.functions import expr
withEventTime\
.withWatermark("event_time", "5 seconds")\
.dropDuplicates(["User", "event_time"])\
.groupBy("User")\
.count()\
.writeStream\
.queryName("pydeduplicated")\
.format("memory")\
.outputMode("complete")\
.start()

In [ ]:
spark.sql('SELECT * FROM pydeduplicated').show()

+----+-----+
|User|count|
+----+-----+
|   a|80855|
|   b|91238|
|   c|77154|
|   g|91674|
|   h|77328|
|   e|96023|
|   f|92056|
|   d|81245|
|   i|92552|
+----+-----+



Arbitrary Stateful Processing

Xử lý trạng thái chỉ khả dụng trong Scala trong Spark 2.2.  Điều này có thể sẽ thay đổi trong tương lai.  Khi thực hiện xử lý trạng thái, bạn có thể muốn thực hiện những việc sau: Tạo cửa sổ dựa trên số lượng của một khóa nhất định.  để thực hiện một số phân tích về sau.  Vào cuối ngày, có hai điều bạn sẽ muốn làm khi thực hiện kiểu xử lý này: Ánh xạ qua các nhóm trong dữ liệu của bạn, thao tác trên từng nhóm dữ liệu và tạo nhiều nhất một hàng cho mỗi nhóm.  API có liên quan cho trường hợp sử dụng này là mapGroupsWithState.  Ánh xạ qua các nhóm trong dữ liệu của bạn, thao tác trên từng nhóm dữ liệu và tạo một hoặc nhiều hàng cho mỗi nhóm.  API có liên quan cho trường hợp sử dụng này là flatMapGroupsWithState.  Khi chúng tôi nói "hoạt động" trên từng nhóm dữ liệu, điều đó có nghĩa là bạn có thể tùy ý cập nhật từng nhóm độc lập với bất kỳ nhóm dữ liệu nào khác.  Điều này có nghĩa là bạn có thể xác định các loại cửa sổ tùy ý không phù hợp với cửa sổ lật hoặc trượt như chúng ta đã thấy trước đó trong chương.  Một lợi ích quan trọng mà chúng tôi nhận được khi thực hiện kiểu xử lý này là kiểm soát việc định cấu hình thời gian chờ ở trạng thái.  Với cửa sổ và hình mờ, rất đơn giản: bạn chỉ cần hết thời gian chờ một cửa sổ khi hình mờ vượt qua cửa sổ bắt đầu.  Điều này không áp dụng cho quá trình xử lý trạng thái tùy ý, vì bạn quản lý trạng thái dựa trên các khái niệm do người dùng xác định.  Do đó, bạn cần thời gian cho trạng thái của mình một cách hợp lý.  Hãy thảo luận thêm về vấn đề này một chút.

Time-Outs

Như đã đề cập trong Chương 21, thời gian chờ xác định bạn nên chờ bao lâu trước khi hết thời gian ở một số trạng thái trung gian.  Thời gian chờ là một tham số chung trên tất cả các nhóm được định cấu hình trên cơ sở từng nhóm.  Thời gian chờ có thể dựa trên thời gian xử lý (GroupStateTimeout.ProcessingTimeTimeout) hoặc thời gian sự kiện (GroupStateTimeout.EventTimeTimeout).  Khi sử dụng thời gian chờ, hãy kiểm tra thời gian chờ trước khi xử lý các giá trị.  Bạn có thể lấy thông tin này bằng cách kiểm tra cờ state.hasTimedOut hoặc kiểm tra xem trình lặp giá trị có trống không.  Bạn cần đặt một số trạng thái (tức là trạng thái phải được xác định, không được xóa) để thời gian chờ được đặt.  Với thời gian chờ dựa trên thời gian xử lý, bạn có thể đặt khoảng thời gian chờ bằng cách gọi GroupState.setTimeoutDuration (chúng ta sẽ xem các ví dụ mã về điều này sau trong phần này của chương).  Thời gian chờ sẽ xảy ra khi đồng hồ đã tăng trước thời lượng đã đặt.  Các đảm bảo được cung cấp bởi khoảng thời gian chờ này với khoảng thời gian D ms như sau: Hết thời gian sẽ không bao giờ xảy ra trước khi thời gian đồng hồ tăng thêm D ms Hết thời gian sẽ xảy ra cuối cùng khi có kích hoạt trong truy vấn (tức là sau  D ms).  Vì vậy, không có giới hạn trên nghiêm ngặt về thời điểm hết thời gian sẽ xảy ra.  Ví dụ: khoảng thời gian kích hoạt của truy vấn sẽ ảnh hưởng đến thời điểm hết thời gian thực sự xảy ra.  Nếu không có dữ liệu trong luồng (cho bất kỳ nhóm nào) trong một thời gian, sẽ không có bất kỳ kích hoạt nào và lệnh gọi hàm thời gian chờ sẽ không xảy ra cho đến khi có dữ liệu.